In [ ]:
[phi_hat, x_filt, a_scale_final]=ar_est_bip_s([1 2 3 4 5]', 2)

phi_hat =

    1.9701   -0.9900


x_filt =

     0
     0
     3
     4
     5


a_scale_final =

    4.3116    1.4040    0.0978
                                              
>> [phi_hat, x_filt, a_scale_final]=ar_est_bip_s([1 2 3 4 5 6 7 8 9]',3)

phi_hat =

    2.9502   -2.9404    0.9900


x_filt =

     0
     0
     0
     4
     5
     6
     7
     8
     9


a_scale_final =

    7.1825    1.4316    0.1562    0.0557

In [1]:
import numpy as np
import robustsp as rsp
rsp.ar_est_bip_s([1,2,3,4],3)

c:\users\computer\desktop\robustsp\robustsp\DependentData\Auxiliary\m_scale.py:15: RuntimeWarning: divide by zero encountered in true_divide
  w_k[x!=0] = rsp.muler_rho1(x[x!=0]/sigma_k) / (x[x!=0] / sigma_k)**2
c:\users\computer\desktop\robustsp\robustsp\DependentData\Auxiliary\m_scale.py:19: RuntimeWarning: invalid value encountered in double_scalars
  if np.abs(sigma_k_plus1/sigma_k-1) > epsilon:


(array([ 0.99    ,  0.960399, -0.99    ]),
 array([0., 0., 0., 4.]),
 [3.594474792773319, 1.397067474856152, 0.08295158847281511, 0.0])

In [1]:
'''
TODO

make a_scale_final nparray of length P instead of list
'''

def ar_est_bip_s(xxx, P):
    '''
    Inputs:
    x   : 1darray, dtype=float. data
    P   : scalar. Autoregressive order
    '''
    x = np.array(xxx)
    N = len(x);
    kap2 = 0.8724286
    phi_grid = np.arange(-.99,.991,.05) # coarse grid search
    fine_grid= np.arange(-.99,.991,.001) # finer grid via polynomial interpolation
    a_bip_sc = np.zeros([len(phi_grid)]) # residual scale for BIP-AR on finer grid 
    a_sc = np.zeros(len(phi_grid)) # residual scale for AR on finer grid

    # The following was introduced so as not to predict based
    # on highly contaminated data in the
    # first few samples.

    x_tran = x[:min(10,int(np.floor(N/2)))]
    sig_x_tran = np.median(np.abs(x-np.median(x)))

    x_tran[np.abs(x_tran)>3*sig_x_tran] = 3*sig_x_tran*np.sign(x_tran[np.abs(x_tran)>3*sig_x_tran])
    x[1:min(10,int(np.floor(N/2)))] = x_tran[1:min(10,int(np.floor(N/2)))]

    phi_hat = np.zeros((P,P))
    x_filt, phi_hat[0,0], a_scale_final = rsp.bip_ar1_s(x,N,phi_grid,fine_grid,kap2)

    npa = lambda x: np.array(x)

    for p in range(1,P):
        for mm in range(len(phi_grid)):

            for pp in range(p):
                phi_hat[p,pp] =\
                phi_hat[p-1,pp]-phi_grid[mm]*phi_hat[p-1,p-pp-1]

            predictor_coeffs =\
            np.array([*phi_hat[p,:p], phi_grid[mm]])

            M = len(predictor_coeffs)
   
            if np.mean(np.abs(np.roots([1, *predictor_coeffs]))<1)==1:
                lambd = rsp.ma_infinity(predictor_coeffs, 0, 100)
                sigma_hat = a_scale_final[0]/np.sqrt(1+kap2*np.sum(lambd**2))
            else:
                sigma_hat = 1.483*np.median(np.abs(x-np.median(x)))

            a = np.zeros(len(x))
            a2= np.zeros(len(x))

            for ii in range(p+1,N):

                xArr = x[ii-1::-1] if ii-M-1 <0 else x[ii-1:ii-M-1:-1]
                aArr = a[ii-1::-1] if ii-M-1 <0 else a[ii-1:ii-M-1:-1]

                a[ii] = compA(x[ii],predictor_coeffs,xArr,aArr,sigma_hat)

                a2[ii] = x[ii] - predictor_coeffs@xArr
            a_bip_sc[mm] = rsp.m_scale(a[p+1:]) # residual scale for BIP-AR
            a_sc[mm] = rsp.m_scale(a2[p+1:]) # residual scale for AR

        # tau-estimate under the BIP-AR(p) and AR(p)
        phi, phi2, temp, temp2, ind_max, ind_max2 = tauEstim(phi_grid, a_bip_sc, fine_grid, a_sc)

        # final estimate minimizes the residual scale of the two
        if temp2<temp:
            ind_max=ind_max2
            temp=temp2
            
        for pp in range(p):
            phi_hat[p,pp] = phi_hat[p-1,pp]-fine_grid[ind_max]*phi_hat[p-1,p-pp-1]

        phi_hat[p,p] = fine_grid[ind_max]

        # final AR(p) tau-scale-estimate depending on phi_hat(p,p)
        if np.mean(np.abs(np.roots([1,*phi_hat[p,:]]))<1) == 1:
            lambd = rsp.ma_infinity(phi_hat[p,:], 0, 100)
            #sigma used for bip-model
            sigma_hat = a_scale_final[0]/np.sqrt(1+kap2*np.sum(lambd**2))
        else:
            sigma_hat = 1.483*np.median(np.abs(x-np.median(x)))

        x_filt = np.zeros(len(x))

        for ii in range(p+1,N):
            xArr = x[ii-1::-1] if ii-M-1 <0 else x[ii-1:ii-M-1:-1]
            aArr = a[ii-1::-1] if ii-M-1 <0 else a[ii-1:ii-M-1:-1]

            a[ii] = x[ii]-phi_hat[p,:p+1]@(xArr-aArr\
            +sigma_hat*rsp.eta(aArr/sigma_hat))

            a2[ii]=x[ii]-phi_hat[p,:p+1]@xArr

        if temp2>temp:
            a_scale_final.append(rsp.m_scale(a[p+1:]))
        else:
            a_scale_final.append(rsp.m_scale(a2[p+1:]))

    phi_hat = phi_hat[p,:] # BIP-AR(P) tau-estimate        

    for ii in range(p+1,N):
        x_filt[ii] = x[ii] - a[ii] + sigma_hat*rsp.eta(a[ii]/sigma_hat)

    return phi_hat, x_filt, a_scale_final


'''
phi_hat =

    1.9701   -0.9900


x_filt =

     0
     0
     3
     4
     5


a_scale_final =

    4.3116    1.4040    0.0978
'''

In [1]:
import numpy as np
import robustsp as rsp

rsp.ar_est_bip_s([1,2,3,4],0)

([], 3.594474792773319)

In [6]:
'''
>> eta([1 2 3]')

ans =

    1.0000
    2.0000
   -0.0000
'''

import numpy as np

def eta(xx,c=1):
    x = np.array(xx)/c
    y = np.zeros(len(x))
    y[np.abs(x)<=2] = x[np.abs(x)<=2]
    
    lnd = lambda x,y: np.logical_and(x,y)
    y[lnd(np.abs(x)<=3,np.abs(x)>2)] = 0.016*x[lnd(np.abs(x)<=3,np.abs(x)>2)]**7\
    -.312*x[lnd(np.abs(x)<=3,np.abs(x)>2)]**5\
    +1.728*x[lnd(np.abs(x)<=3,np.abs(x)>2)]**3\
    -1.944*x[lnd(np.abs(x)<=3,np.abs(x)>2)]
    
    y= c*y
    
    return y

eta([1,2,3])

array([ 1.00000000e+00,  2.00000000e+00, -6.21724894e-15])

In [4]:
import numpy as np
a = [1,2,3,1,2,3,1,2,3]
a = np.array(a)
a[5:2:-1]

array([3, 2, 1])

In [6]:
a[a<2]
len(np.array(0).shape)

0

In [64]:
a= np.zeros((3,3))
np.isscalar([1,2,3])
rsp.eta([0.3413,0.3310])

array([0.3413, 0.331 ])

In [128]:
a=[1,2,3,4,5]
M = 2
'''
2,1
3,2
4,3
'''
for i in range(2,5):
    if i-M-1 <0:
        print(a[i-1::-1])
    else:
        print(a[i-1:i-M-1:-1])

[2, 1]
[3, 2]
[4, 3]


In [148]:
3 \
- np.array([1.9701,-0.9900]) \
@ (np.array([2,1])\
    -np.array([0,0])+1.483*\
    rsp.eta(np.array([0,0])/1.483))


0.04980000000000029

In [1]:
import numpy as np
import robustsp as rsp

def compA(xII,pred,xArr,aArr,sig):
    return xII \
- np.array(pred) @\
(np.array(xArr)-np.array(aArr)+sig*rsp.eta(np.array(aArr)/sig))
    
    
def tauEstim(phi_grid, a_bip_sc, fine_grid, a_sc):    
    poly_approx = np.polyfit(phi_grid, a_bip_sc, 5) # polynomial approximation of residual scale for BIP-AR(p) tau-estimates
    a_interp_scale = np.polyval(poly_approx, fine_grid) # interpolation of  residual scale for BIP-AR(p) tau-estimates to fine grid
    poly_approx2= np.polyfit(phi_grid, a_sc, 5) # polynomial approximation of  residual scale for AR(p) tau-estimates
    a_interp_scale2= np.polyval(poly_approx2, fine_grid) # interpolation of  residual scale for AR(p) tau-estimates to fine grid
   
    temp = np.min(a_interp_scale)
    ind_max = np.argmin(a_interp_scale)
    phi = -fine_grid[ind_max] # tau-estimate under the BIP-AR(p)

    temp2 = np.min(a_interp_scale2)
    ind_max2 = np.argmin(a_interp_scale2)
    phi2=-fine_grid[ind_max2] # tau-estimate under the AR(p)  
    return phi, phi2, temp, temp2, ind_max, ind_max2                   

In [ ]:
a = np.array([[1,2,3],[4,5,6],[7,8,9]])
a[1:0:]